#### FastAPI

##### Create Python Virtual Environment and install all dependencies as follows ######

create venv using command: python -m venv venvname

activate venv: venvname\Scripts\Activate

install dependencies: pip install -r requirements.txt

run using: uvicorn main:app --reload

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(filepath_or_buffer=url,header=None,sep=',',names=names)

# Split-out validation dataset
array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

classifier = LogisticRegression()
classifier.fit(X_train,y_train)

#save the model to disk
pickle.dump(classifier, open('LRClassifier.pkl', 'wb'))

#load the model from disk
loaded_model = pickle.load(open('LRClassifier.pkl', 'rb'))

result = loaded_model.score(X_test, y_test)
print(result)

0.9666666666666667


In [5]:
from fastapi import FastAPI, File, Form, UploadFile
from pydantic import BaseModel
import pickle
import numpy as np
import pandas as pd
from io import StringIO

app = FastAPI()

class IrisSpecies(BaseModel):
    sepal_length: float 
    sepal_width: float 
    petal_length: float 
    petal_width: float

@app.post('/predict')
async def predict_species(iris: IrisSpecies):
    data = iris.dict()
    loaded_model = pickle.load(open('LRClassifier.pkl', 'rb'))
    data_in = [[data['sepal_length'], data['sepal_width'], data['petal_length'], data['petal_width']]]
    prediction = loaded_model.predict(data_in)
    probability = loaded_model.predict_proba(data_in).max()
            
    return {
        'prediction': prediction[0],
        'probability': probability
    }

@app.post("/files/")
async def create_file(file: bytes = File(...), token: str = Form(...)):
    s=str(file,'utf-8')
    data = StringIO(s)
    df=pd.read_csv(data)
    print(df)
    #return df
    return {
        "file": df,
        "token": token,
    }

In [12]:
# main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}

In [ ]:
uvicorn src.main:app 
    